On va prédire en se servant de notre modèle

In [1]:
import joblib
from requests import get
import requests
from bs4 import BeautifulSoup
import re
import time
import polars as pl
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier

Récupération de mon modèle

In [ ]:
modele_charge = joblib.load('mon_modele_lol.pkl')
#prediction = modele_charge.predict("ce que je veux prédire")
print("Modèle chargé et prêt.")

roles_mapping = {
    0: "combattant",
    1: "tirreur",
    2: "mage",
    3: "support",
    4: "assassin",
    5: "tank"
}
print("Le modèle de traduction est prêt.")

class Statchamp:
    def __init__(self, AttackRange, MoveSpeed, HP, HPperLevel, hpregen, 
                 hpregenperlevel, Armor, ArmorperLevel, AttackDamage, 
                 AttackDamageperLevel, attackspeed, attackspeedperlevel, 
                 Spellblock, spellblockperlevel, Prv_attack, Prv_defense, Prv_magic):
        
        # Stockage des données brutes
        self.raw_data = {
            "AttackRange": [AttackRange],
            "MoveSpeed": [MoveSpeed],
            "HP": [HP],
            "HPperLevel": [HPperLevel],
            "hpregen": [hpregen],
            "hpregenperlevel": [hpregenperlevel],
            "Armor": [Armor],
            "ArmorperLevel": [ArmorperLevel],
            "AttackDamage": [AttackDamage],
            "AttackDamageperLevel": [AttackDamageperLevel],
            "attackspeed": [attackspeed],
            "attackspeedperlevel": [attackspeedperlevel],
            "Spellblock": [Spellblock],
            "spellblockperlevel": [spellblockperlevel],
            "Prv_attack": [Prv_attack],
            "Prv_defense": [Prv_defense],
            "Prv_magic": [Prv_magic]
        }

    def get_dataframe(self):
        """Transforme les données, calcule les stats au lvl 10 et nettoie les colonnes inutiles"""
        df = pl.DataFrame(self.raw_data)
        
        # 1. Calcul des valeurs au niveau 10
        df = df.with_columns(
            (pl.col("Spellblock") + pl.col("spellblockperlevel") * 9).alias("Spellblock_lvl_10"),
            (pl.col("attackspeed") + pl.col("attackspeedperlevel") * 9).alias("attackspeed_lvl_10"),
            (pl.col("AttackDamage") + pl.col("AttackDamageperLevel") * 9).alias("AttackDamage_lvl_10"),
            (pl.col("Armor") + pl.col("ArmorperLevel") * 9).alias("Armor_lvl_10"),
            (pl.col("hpregen") + pl.col("hpregenperlevel") * 9).alias("hpregen_lvl_10"),
            (pl.col("HP") + pl.col("HPperLevel") * 9).alias("hp_lvl_10")
        )

        # 2. Suppression des colonnes "per level" devenues inutiles
        cols_to_drop = [
            "spellblockperlevel", 
            "attackspeedperlevel", 
            "AttackDamageperLevel", 
            "ArmorperLevel", 
            "hpregenperlevel", 
            "HPperLevel"
        ]
        df = df.drop(cols_to_drop)
        
        return df
print("La classe du champion est prête.")

Modèle chargé et prêt.


Création du champ test

In [20]:
# 1. Créer un DataFrame avec les caractéristiques du nouveau champion
nouveau_champ = pl.DataFrame({
    "AttackRange": [175],          # Portée
    "MoveSpeed": [340],            # Vitesse de déplacement
    "HP": [650],                   # Points de vie
    "HPperLevel": [114],            # PV par niveau
    "hpregen": [3.5],              # Régénération PV
    "hpregenperlevel": [0.6],      # Régénération PV par niveau
    "Armor": [30],                 # Armure
    "ArmorperLevel": [3.5],        # Armure par niveau
    "AttackDamage": [98],          # Dégâts d'attaque
    "AttackDamageperLevel": [3],   # Dégâts par niveau
    "attackspeed": [0.625],        # Vitesse d'attaque
    "attackspeedperlevel": [2.5],  # % vitesse d'attaque par niveau
    "Spellblock": [303],            # Résistance magique
    "spellblockperlevel": [0.5],   # Résistance magique par niveau
    "Prv_attack": [5],             # Note d'attaque (ex: 1 à 10)
    "Prv_defense": [5],            # Note de défense
    "Prv_magic": [5]               # Note de magie
})

#2. Calculer les valeurs au niveau 10
nouveau_champ = nouveau_champ.with_columns(
    (pl.col("Spellblock") + pl.col("spellblockperlevel") * 9).alias("Spellblock_lvl_10"),
    (pl.col("attackspeed") + pl.col("attackspeedperlevel") * 9).alias("attackspeed_lvl_10"),
    (pl.col("AttackDamage") + pl.col("AttackDamageperLevel") * 9).alias("AttackDamage_lvl_10"),
    (pl.col("Armor") + pl.col("ArmorperLevel") * 9).alias("Armor_lvl_10"),
    (pl.col("hpregen") + pl.col("hpregenperlevel") * 9).alias("hpregen_lvl_10"),
    (pl.col("HP") + pl.col("HPperLevel") * 9).alias("hp_lvl_10"))

nouveau_champ = nouveau_champ.drop(["spellblockperlevel", "attackspeedperlevel", "AttackDamageperLevel", "ArmorperLevel", "hpregenperlevel", "HPperLevel"])

# 3. On sélectionne les colonnes finales dans le bon ordre (celui du X_train)
cols_entrainement = ['AttackRange', 'MoveSpeed', 'HP', 'hpregen', 'Armor', 'AttackDamage', 'attackspeed', 'Spellblock', 'Prv_attack', 'Prv_defense', 'Prv_magic', 'Spellblock_lvl_10', 'attackspeed_lvl_10', 'AttackDamage_lvl_10', 'Armor_lvl_10', 'hpregen_lvl_10', 'hp_lvl_10'] # Exemple, mets ta vraie liste
X_nouveau = nouveau_champ.select(cols_entrainement)

# 4. Prédiction
resultat = modele_charge.predict(X_nouveau)

print(f"Le rôle prédit est : {resultat[0]}")

Le rôle prédit est : 0


C:\Users\Marc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


Traduction

In [ ]:
role_texte = roles_mapping[resultat[0]]
print(f"L'IA pense que c'est un {role_texte}")

L'IA pense que c'est un combattant


Création d'une classe " nouveau champion"

In [ ]:


# 1. Création du champion
nouveau_champ = Statchamp(
    AttackRange=175,
    MoveSpeed=340,
    HP=650,
    HPperLevel=114,
    hpregen=3.5,
    hpregenperlevel=0.6,
    Armor=30,
    ArmorperLevel=3.5,
    AttackDamage=98,
    AttackDamageperLevel=3,
    attackspeed=0.625,
    attackspeedperlevel=2.5, # Attention : Vérifie si c'est des % (0.025) ou brut (2.5)
    Spellblock=303,
    spellblockperlevel=0.5,
    Prv_attack=5,
    Prv_defense=5,
    Prv_magic=5
)

# 2. Récupération du tableau propre
df_pret = nouveau_champ.get_dataframe()

# 3. Affichage
print(df_pret)

shape: (1, 17)
┌─────────────┬───────────┬─────┬─────────┬───┬─────────────┬─────────────┬────────────┬───────────┐
│ AttackRange ┆ MoveSpeed ┆ HP  ┆ hpregen ┆ … ┆ AttackDamag ┆ Armor_lvl_1 ┆ hpregen_lv ┆ hp_lvl_10 │
│ ---         ┆ ---       ┆ --- ┆ ---     ┆   ┆ e_lvl_10    ┆ 0           ┆ l_10       ┆ ---       │
│ i64         ┆ i64       ┆ i64 ┆ f64     ┆   ┆ ---         ┆ ---         ┆ ---        ┆ i64       │
│             ┆           ┆     ┆         ┆   ┆ i64         ┆ f64         ┆ f64        ┆           │
╞═════════════╪═══════════╪═════╪═════════╪═══╪═════════════╪═════════════╪════════════╪═══════════╡
│ 175         ┆ 340       ┆ 650 ┆ 3.5     ┆ … ┆ 125         ┆ 61.5        ┆ 8.9        ┆ 1676      │
└─────────────┴───────────┴─────┴─────────┴───┴─────────────┴─────────────┴────────────┴───────────┘


Test de a prédiction 

In [7]:
#Prédiction
resultat = modele_charge.predict(df_pret)

#print(f"Le rôle prédit est : {resultat[0]}")
#Traduction
role_texte = roles_mapping[resultat[0]]

print(f"L'IA pense que c'est un {role_texte}")


L'IA pense que c'est un combattant


C:\Users\Marc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(
